In [1]:
import pdfplumber
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from bs4 import BeautifulSoup
import requests
import urllib.request
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import mplfinance as mpf
import statistics

In [2]:
# define the name of the Dataset(NA: assuming a problem with the extract strategy, we predict a colum that will be deleted )
ncolumns=['Lote','Sexo','Cantidad','Peso Total','Peso Promedio','Procedencia','NA','Hora Entrada','Precio Base','Precio Final','O','O2']

# create the dataframe with the corresponding columns
df=pd.DataFrame(columns=ncolumns)

# Define the download range
start_pdf = 1683
end_pdf = 1683

#Create a loop for read pdf and extract the tables.
pdf_template="C://Users//Personal//Desktop//PDF Subacasanare//{pdf_number}.pdf"
for pdf_number in range(start_pdf, end_pdf + 1):
    pdf_ad = pdf_template.format(pdf_number=pdf_number)
    pdf1=pdfplumber.open(pdf_ad)
    for i in range(len(pdf1.pages)-1):
        page1=pdf1.pages[i]
        table1=pd.DataFrame(page1.extract_table(table_settings={"vertical_strategy": "text"})).iloc[2:,:11]
        table1.rename(columns={0:'Lote',1:'Sexo',2:'Cantidad',3:'Peso Total',4:'Peso Promedio',5:'Procedencia',6:'NA',7:'Hora Entrada',8:'Precio Base',9:'Precio Final',10:'O',11:'O2'},inplace=True)
        #Obtain the Date extracting the first 10 characters of each page and add to the Dataframe
        text1=page1.extract_text()
        table1['Fecha']=text1[0:10]
        table1['Fecha']=pd.to_datetime(table1['Fecha']) 
        words = text1.split()
        extracted_text=''
        for i in range(len(words)):
            if words[i] == 'MARTILLO':
                extracted_text += ' '.join(words[i + 1: i + 5])
            
        table1['Martillo']=extracted_text
        df=pd.concat([df, table1], ignore_index=True)
        
        
        
df
#put the date colum first
df = df.reindex(columns=['Fecha','Martillo','Lote','Sexo','Cantidad','Peso Total','Peso Promedio','Procedencia','NA','Hora Entrada','Precio Base','Precio Final','O','O2'])
#The column NA its for the cities that have a long name and the extract estrategy take like two columns (NA its the 2d column), 
# all the cities arent that long, for that reason some 'Hora Entrada' values are mislocated in NA,to avoid this, if string number values
#are detected in NA colum thh values are displaced to the right 
mask = df['NA'].str.match(r'^\d+')
mask = mask.fillna(False)
df.loc[mask, 'NA':] = df.loc[mask, 'NA':].shift(1, axis=1)
df.drop('NA', axis=1, inplace=True)
df['Precio Final']=df['Precio Final'].astype(str)
df['Precio Final']=df['Precio Final'].str.replace('.','')
df['Precio Base']=df['Precio Base'].astype(str)
df['Precio Base']=df['Precio Base'].str.replace('.','')
df=df.loc[df['Precio Final'].str.isnumeric()&df['Precio Base'].str.isnumeric()]
df['Peso Total']=df['Peso Total'].astype(str)
df['Peso Total']=df['Peso Total'].str.replace('.','')
df['Peso Promedio'] = df['Peso Promedio'].astype(str)
df['Peso Promedio'] = df['Peso Promedio'].str.replace('.', '')
df['Lote']=df['Lote'].astype(int)
df['Cantidad']=df['Cantidad'].astype(int)
df['Precio Final'] = df['Precio Final'].astype(int)
df['Precio Base'] = df['Precio Base'].astype(int)
df['Sexo']=df['Sexo'].astype(str)
df['Peso Total']=df['Peso Total'].astype(int)
df['Peso Promedio']=df['Peso Promedio'].astype(int)
df['Dia'] = df['Fecha'].dt.day_name()
# pd.set_option('display.max_rows',None)
# pd.set_option('display.max_columns',None)
df




C:\Users\Personal\AppData\Local\Temp\ipykernel_10504\22301954.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Precio Final']=df['Precio Final'].str.replace('.','')
C:\Users\Personal\AppData\Local\Temp\ipykernel_10504\22301954.py:48: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Precio Base']=df['Precio Base'].str.replace('.','')
C:\Users\Personal\AppData\Local\Temp\ipykernel_10504\22301954.py:51: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Peso Total']=df['Peso Total'].str.replace('.','')
C:\Users\Pers

,Fecha,Martillo,Lote,Sexo,Cantidad,Peso Total,Peso Promedio,Procedencia,Hora Entrada,Precio Base,Precio Final,O,O2,Dia
0,2023-11-23,NELIDA OMAIRA GOMEZ DUE?AS,1,VC,5,1874,375,YOPAL,10:49 am,4700,4950,1 BAJA CONDICION,CORPORAL,Thursday
1,2023-11-23,NELIDA OMAIRA GOMEZ DUE?AS,2,VC,5,2080,416,YOPAL,10:51 am,5200,5250,,,Thursday
2,2023-11-23,NELIDA OMAIRA GOMEZ DUE?AS,3,MC,13,3406,262,YOPAL,10:57 am,6600,6750,1 CON INFLAMACIO,N EN NARIZ,Thursday
3,2023-11-23,NELIDA OMAIRA GOMEZ DUE?AS,4,ML,11,1932,176,OROCUE,11:35 am,8600,8750,,,Thursday
4,2023-11-23,NELIDA OMAIRA GOMEZ DUE?AS,5,MC,8,2094,262,OROCUE,11:58 am,6600,7600,7 NOVILLOS,,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2023-11-23,NELIDA OMAIRA GOMEZ DUE?AS,147,VG,7,2502,357,NUNCHIA,10:12 am,6000,6300,4VG Y 3N,G,Thursday
146,2023-11-23,NELIDA OMAIRA GOMEZ DUE?AS,148,ML,5,1002,200,OROCUE,10:17 am,8200,8550,,,Thursday
147,2023-11-23,NELIDA OMAIRA GOMEZ DUE?AS,149,VC,8,3042,380,MANI,11:40 am,5700,6000,,,Thursday
148,2023-11-23,NELIDA OMAIRA GOMEZ DUE?AS,150,ML,14,3221,230,YOPAL,05:07 pm,8200,8550,,,Thursday


In [5]:
df.to_csv("C://Users//Personal//Downloads//SubacasanareHora.csv")
df['Procedencia'].unique()

array(['NUNCHIA', 'YOPAL', 'AGUAZUL', 'SAN LUIS DE', 'OROCUE', 'TAMARA',
       'OROCU', 'HATO COROZ', 'TRINIDAD', 'PAZ DE ARIP', 'NUNCH?A',
       'TRNIDAD', 'PAZ DE ARIPORO', 'HATO COROZAL', 'SAN LUIS DE PALENQ',
       'PAJARITO(BOYACA)', 'MANI'], dtype=object)

In [3]:
# # filter the data, wether i know what value must be taken the data, i take advantage for delete the missextract values. 
df['Procedencia'].replace({'SN LUIS DE PAENQUE':'SAN LUIS DE PALENQUE','SAN LUIS NDE PALEN':'SAN LUIS DE PALENQUE','SAN LUIS PALENQUE':'SAN LUIS DE PALENQUE','SAN LUISE DE PALEN':'SAN LUIS DE PALENQUE','SAN LUIS DE  PALEN':'SAN LUIS DE PALENQUE','SAN LUIS DE P?LENQ':'SAN LUIS DE PALENQUE','SAN LUIS DE':'SAN LUIS DE PALENQUE','SAN LUIS DE PALENQ':'SAN LUIS DE PALENQUE', 'PAZ DE ARIP':'PAZ DE ARIPORO','HATO COROZ':'HATO COROZAL','AGUAUZL':'AGUAZUL','?AGUAZUL':'AGUAZUL','TAME':'ARAUCA','TAME ARAUCA':'ARAUCA','TAME( ARAUCA)':'ARAUCA','TAME(ARAUCA)':'ARAUCA','TAME(ARAUCA9':'ARAUCA', 'SARAVENA':'ARAUCA','SARAVENA  ARAUCA':'ARAUCA','SARAVENA( ARAUCA)':'ARAUCA','SARAVENA(ARAUCA)':'ARAUCA','ARAUCA(SARAVENA)':'ARAUCA','ARAUCa':'ARAUCA','PUERTO GAITAN(META':'META','CUMARAL(META)':'META','META(CABUYARO)':'META','FORTUL ARAUCA':'ARAUCA','PUERTO RONDON':'ARAUCA','FORTUL(ARAUCA)':'ARAUCA','HATOCOROZAL':'HATO COROZAL','LABRANZAGRANDE BO':'LABRANZAGRANDE','LABRAZAGRANDE':'LABRANZAGRANDE','MONTEREY':'MONTERREY','MOTERREY':'MONTERREY','OROCU':'OROCUE','OROCUES':'OROCUE','PAZ DE  ARIPORO':'PAZ DE ARIPORO','PAZ DE A RIPORO':'PAZ DE ARIPORO','PAZ DE AIPORO':'PAZ DE ARIPORO','PAZ DE ARIPOIRO':'PAZ DE ARIPORO','PAZ DE ARIPORO (MO':'PAZ DE ARIPORO','PAZ DE ARIPPORO':'PAZ DE ARIPORO','TAURAMEA':'TAURAMENA','?TAURAMENA':'TAURAMENA','TRINIDA D':'TRINIDAD','TRINIDAD BOCAS DEL':'TRINIDAD','TRINIDAD( BOCAS DE':'TRINIDAD','TRINIDAD(BOCAS DEL':'TRINIDAD','TRNIDAD':'TRINIDAD','YOAPAL':'YOPAL','YOAPL':'YOPAL','YOPSL':'YOPAL','?YOPAL':'YOPAL','YOPÁL':'YOPAL','mani':'MANI'}, inplace=True)
miss=(df['Precio Base']<(20000))&(df['Precio Final']<(20000))
# & (df['Hora Entrada'].str.endswith('m')&((df['Procedencia']=='SAN LUIS DE PALENQUE')|(df['Procedencia']=='VILLANUEVA')|(df['Procedencia']=='ARAUCA')|(df['Procedencia']=='MONTERREY')|(df['Procedencia']=='TAME')|df['Procedencia']=='TAMARA')|(df['Procedencia']=='HATO COROZAL')|(df['Procedencia']=='PAZ DE ARIPORO')|(df['Procedencia']=='MANI')|(df['Procedencia']=='AGUAZUL')|(df['Procedencia']=='PORE')|(df['Procedencia']=='TRINIDAD')|(df['Procedencia']=='NUNCHIA')|(df['Procedencia']=='OROCUE')|(df['Procedencia']=='YOPAL'))
df=df.loc[miss]
# df['V% Precio']=(df['Precio Final']/df['Precio Base'])-1
# # pd.set_option('display.max_rows',None)
# # pd.set_option('display.max_columns',None)
df

,Fecha,Martillo,Lote,Sexo,Cantidad,Peso Total,Peso Promedio,Procedencia,Hora Entrada,Precio Base,Precio Final,O,O2,Dia
0,2023-11-21,NELIDA OMAIRA GOMEZ DUE?AS,1,VC,1,376,376,YOPAL,09:38 am,4000,4500,,NaN,Tuesday
1,2023-11-21,NELIDA OMAIRA GOMEZ DUE?AS,2,MC,1,420,420,YOPAL,09:41 am,6800,7200,,NaN,Tuesday
2,2023-11-21,NELIDA OMAIRA GOMEZ DUE?AS,3,VC,2,948,474,YOPAL,09:32 am,5200,5650,,NaN,Tuesday
3,2023-11-21,NELIDA OMAIRA GOMEZ DUE?AS,4,ML,1,198,198,YOPAL,09:43 am,8500,8900,,NaN,Tuesday
4,2023-11-21,NELIDA OMAIRA GOMEZ DUE?AS,5,ML,1,240,240,MONTERREY,09:56 am,7500,8100,,NaN,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,2023-11-21,NELIDA OMAIRA GOMEZ DUE?AS,280,MG,3,1660,553,MANI,01:30 pm,7700,8000,,NaN,Tuesday
273,2023-11-21,NELIDA OMAIRA GOMEZ DUE?AS,281,VG,1,443,443,YOPAL,03:19 pm,6000,6450,INFLAMACION,NaN,Tuesday
274,2023-11-21,NELIDA OMAIRA GOMEZ DUE?AS,282,VG,1,400,400,YOPAL,05:08 pm,5900,6150,,NaN,Tuesday
275,2023-11-21,NELIDA OMAIRA GOMEZ DUE?AS,283,NG,1,408,408,NUNCHIA,05:09 pm,6300,6550,,NaN,Tuesday


In [5]:
df.to_csv("C://Users//Personal//Downloads//1682.csv")

In [3]:
LabelS=df['Sexo'].unique().astype(str)
LabelS
X=df[['Sexo','Peso Promedio','Precio Final','Cantidad']]
X
result_table = pd.DataFrame()

# Iterate over each asset label
for i in LabelS:
    # Get the prices for the current asset
    Peso_Promedio= X[X['Sexo']==i] ['Peso Promedio']
    Precio_Final= X[X['Sexo']==i]['Precio Final']
    cantidad=X[X['Sexo']==i]['Cantidad']
    # Calculate the range and mean
    Peso_range = (min(Peso_Promedio), max(Peso_Promedio) + 1)
    Peso_mean = Peso_Promedio.mean()
    Precio_range=(min(Precio_Final), max(Precio_Final) + 1)
    Precio_mean = Precio_Final.mean()
    Cantidad= sum(cantidad)
    # Add a new row to the result table
    result_table = result_table.append({
        'Sexo': i,
        'Rango Peso ': Peso_range,
        'Media Peso ': round(Peso_mean),
        'Rango Precio':Precio_range,
        'Media Precio':round(Precio_mean),
        'Cantidad':Cantidad
    }, ignore_index=True)

    
result_table

C:\Users\Personal\AppData\Local\Temp\ipykernel_10504\1514354318.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_table = result_table.append({
C:\Users\Personal\AppData\Local\Temp\ipykernel_10504\1514354318.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_table = result_table.append({
C:\Users\Personal\AppData\Local\Temp\ipykernel_10504\1514354318.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_table = result_table.append({
C:\Users\Personal\AppData\Local\Temp\ipykernel_10504\1514354318.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_table = result_table.append({
C:\Users\Personal\AppData\Lo

,Sexo,Rango Peso,Media Peso,Rango Precio,Media Precio,Cantidad
0,VC,"(303, 430)",366,"(4850, 6601)",5797,147
1,MC,"(214, 481)",321,"(6500, 8501)",7767,527
2,ML,"(108, 249)",180,"(6700, 10051)",8871,677
3,NC,"(223, 346)",287,"(6100, 6951)",6511,150
4,HL,"(143, 250)",194,"(5950, 7351)",6425,101
5,VG,"(300, 485)",408,"(5650, 6401)",6050,77
6,NG,"(338, 365)",354,"(6550, 7201)",6740,25
7,MG,"(550, 553)",551,"(7850, 8001)",7925,30


In [4]:
result_table.to_csv("C://Users//Personal//Downloads//1683.csv")


In [ ]:
pd.read_csv('')